## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BBBBSSBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1458
+-----------------+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+--------------------+------------------+-----------+--------------------+------+--------------------+------+
|          coeff_1|           coeff_2|           coeff_3|           coeff_4|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|               res_4|          res_mean|    station|           total_var|undefs|              vector|  year|
+-----------------+------------------+------------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+--------------------+------------------+-----------+--------------------+------+--------------------+------+
|9171.110131269304| 2087.753769407183|  

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1,coeff_2,coeff_3,coeff_4'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1,coeff_2,coeff_3,coeff_4 FROM weather


,station,year,coeff_1,coeff_2,coeff_3,coeff_4
0,USC00306184,1977.0,9171.110131,2087.753769,2267.781398,480.251809
1,USC00300785,1952.0,6367.604742,-61.470826,210.861473,-180.253673
2,USC00304912,1994.0,4894.513796,1061.574594,-797.829716,466.551624
3,USC00308248,1978.0,7370.677472,545.421949,1217.108838,810.103121
4,USC00306623,1979.0,7870.919145,1479.027332,-994.701490,-865.182822


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,CA006150815,US1NYOD0014,US1NYOD0023,US1NYOG0046,USC00300015,USC00300018,USC00300368,USC00300379,USC00300424,USC00300500,...,USC00309248,USC00309389,USC00309480,USC00309544,USW00014717,USW00014750,USW00014771,USW00014772,USW00014774,USW00094794
year,,,,,,,,,,,,,,,,,,,,,
1900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1927.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1928.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1929.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1930.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

In [9]:
def rms_explained(coeff):
    global pdf
    year_station_table=pdf.pivot(index='year', columns='station', values=coeff)
    mean_by_year=np.nanmean(year_station_table,axis=1)
    mean_by_station=np.nanmean(year_station_table,axis=0)
    tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
    tbl_minus_station = year_station_table-mean_by_station
    trms = RMS(year_station_table)
    tms = RMS(tbl_minus_station)
    tmy = RMS(tbl_minus_year)
    fes = 100.-tms/trms*100.
    fey = 100.-tmy/trms*100.
    print "\n\n** {0} **".format(coeff.replace('_',' '))
    print "* Total MS = {0:.2f}".format(trms)
    print "* MS removing mean-by-station = {0:.2f}, fraction explained = {1:.2f}".format(tms,fes)
    print "* MS removing mean-by-year = {0:.2f}, fraction explained = {1:.2f}".format(tmy,fey)

In [10]:
coeffs = ('coeff_1','coeff_2','coeff_3','coeff_4')
for c in coeffs:
    rms_explained(c)



** coeff 1 **
* Total MS = 3442.13
* MS removing mean-by-station = 1604.65, fraction explained = 53.38
* MS removing mean-by-year = 1800.90, fraction explained = 47.68


** coeff 2 **
* Total MS = 1040.34
* MS removing mean-by-station = 842.54, fraction explained = 19.01
* MS removing mean-by-year = 558.01, fraction explained = 46.36


** coeff 3 **
* Total MS = 792.11
* MS removing mean-by-station = 662.58, fraction explained = 16.35
* MS removing mean-by-year = 448.68, fraction explained = 43.36


** coeff 4 **
* Total MS = 640.63
* MS removing mean-by-station = 613.75, fraction explained = 4.20
* MS removing mean-by-year = 442.22, fraction explained = 30.97


In [11]:
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station
trms = RMS(year_station_table)
tms = RMS(tbl_minus_station)
tmy = RMS(tbl_minus_year)
fes = 100.-tms/trms*100.
fey = 100.-tmy/trms*100.

print 'Total RMS                   = ',trms
print "RMS removing mean-by-station= {0}, fraction explained={1:.2f}".format(tms,fes)
print "RMS removing mean-by-year   = {0}, fraction explained={1:.2f}".format(tmy,fey)

Total RMS                   =  1040.34026382
RMS removing mean-by-station= 842.544090651, fraction explained=19.01
RMS removing mean-by-year   = 558.006256685, fraction explained=46.36


In [12]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 1040.34026382
0 after removing mean by year    = 558.006256685
0 after removing mean by stations= 516.180095954
1 after removing mean by year    = 514.816291317
1 after removing mean by stations= 514.547751831
2 after removing mean by year    = 514.458206892
2 after removing mean by stations= 514.424844249
3 after removing mean by year    = 514.41195468
3 after removing mean by stations= 514.406878607
4 after removing mean by year    = 514.40485465
4 after removing mean by stations= 514.404040717
